In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
from pathlib import Path
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from scipy.special import softmax as softmax
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralCoclustering
from dppy.finite_dpps import FiniteDPP

from dataloader.builder import build_dataset
from experiment_setup import build_estimator
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from analysis.metrics import uq_ndcg

from model.cnn import SimpleConv, MediumConv, StrongConv
from model.trainer import Trainer, EnsembleTrainer

# torch.cuda.set_device(1)
#

In [ ]:

plt.rcParams['figure.facecolor'] = 'white'
config = {
    'use_cuda': True,
    'seed': 1,
    
    'nn_runs': 150,
    'patience': 5,
    'dropout_uq': 0.5,
    
    'n_models': 10, 
    
    # 'dataset': 'mnist',
    'dataset': 'cifar_10',
   
    'model_class': StrongConv,
    'train_samples': 45_000,
    'epochs': 50,
    'batch_size': 256,
    'log_interval': 150,
    'lr': 1e-2,
    'num_classes': 9
}
restore_model = True


In [ ]:
dataset = build_dataset(config['dataset'], val_size=10_000)
x_train, y_train = dataset.dataset('train')
x_val, y_val = dataset.dataset('val')

def cut_class(x, y, class_num):
    new_x = x[np.where(y!=class_num)]
    new_y = y[np.where(y!=class_num)]
    ood = x[np.where(y==class_num)]
    return new_x, new_y, ood

x_train, y_train, x_ood = cut_class(x_train, y_train, '9')
x_val, y_val, _ = cut_class(x_val, y_val, '9')



In [ ]:
def scale(images):
    return (images - 128) / 128
x_train = scale(x_train)
x_val = scale(x_val)
x_ood = scale(x_ood)

In [ ]:
if config['dataset'] == 'mnist':
    input_shape = (-1, 1, 28, 28)
elif config['dataset'] == 'cifar_10':
    input_shape = (-1, 3, 32, 32)
x_train = x_train.reshape(input_shape)
x_val = x_val.reshape(input_shape)
x_ood = x_ood.reshape(input_shape)

y_train = y_train.astype('long').reshape(-1)
y_val = y_val.astype('long').reshape(-1)



In [ ]:
train_samples = config['train_samples']
epochs = config['epochs']
val_samples = 2000
patience = config['patience']

idxs = np.random.choice(len(x_train), train_samples, replace=False)
train_set = (x_train[idxs], y_train[idxs])

idxs = np.random.choice(len(x_val), val_samples, replace=False)
val_set = (x_val[idxs], y_val[idxs]) 
print(train_set[0].shape)
print(val_set[0].shape)

In [ ]:
model_class = config['model_class'] 
model = model_class(config['num_classes'])

model_dir = Path('experiments') / 'data'
model_path = model_dir / f"model_{config['dataset']}_{config['model_class'].__name__}.pt"

trainer = Trainer(model, dropout_train=config['dropout_train'])

if restore_model and os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
else:
    trainer.fit(
        train_set, val_set , epochs=epochs, verbose=True, patience=patience)
    torch.save(model.state_dict(), model_path)

In [ ]:
trainer.predict(val_set[0][:5])

In [ ]:
mask_type = 'dpp'
masks = build_masks([mask_type])
mask = masks[mask_type]
estimator = build_estimator(
    'bald_masked', trainer, nn_runs=config['nn_runs'], dropout_mask=mask,
    dropout_rate=config['dropout_uq'], num_classes=config['num_classes'])

In [ ]:
estimator.estimate(val_set[0][:5])

In [ ]:
corr = mask.layer_correlations[1]

In [ ]:
plt.figure(figsize=(20, 14))
sns.heatmap(corr)

In [ ]:
for n in range(2, 12):
    plt.figure(figsize=(14, 12))
    plt.title(n)
    model = SpectralCoclustering(n_clusters=n, random_state=0)
    model.fit(corr)
    fit_data = corr[np.argsort(model.row_labels_)]
    fit_data = fit_data[:, np.argsort(model.column_labels_)]
    sns.heatmap(fit_data)
    plt.show()

In [ ]:
model = SpectralCoclustering(n_clusters=5, random_state=0)
model.fit(corr)
corr_5 = corr[np.argsort(model.row_labels_)]
corr_5 = corr_5[:, np.argsort(model.column_labels_)]

In [ ]:
dpp = FiniteDPP('correlation', **{'K': corr})

In [ ]:
[dpp.sample_exact() for _ in range(100)]

In [ ]:
[len(sample) for sample in dpp.list_of_samples]

In [ ]:
def rank(dpp):
    N = dpp.eig_vecs.shape[0]
    tol = np.max(dpp.L_eig_vals) * N * np.finfo(np.float).eps
    rank = np.count_nonzero(dpp.L_eig_vals > tol)
    return rank

In [ ]:
dpp.sample_exact_k_dpp(1)
rank(dpp)

In [ ]:
dpp.sample_exact()
dpp.list_of_samples